In [ ]:
%pylab inline

import csv
import imageio
import numpy as np
import os

----
## Load cameras + point cloud from OmniPhotos cache

In [ ]:
dataset_path = r"G:\OmniPhotos\Data\KobeGarden6"
cache_path = dataset_path + r"\Cache\180-4k-2k-DIS-bugfix"
cameras_csv = cache_path + r"\Cameras.csv"
cameras = []

with open(cameras_csv, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        cameras.append({
            'frame': int(row[0]),
            'K': np.mat([float(e) for e in row[1:10]]).reshape([3,3]),
            'R': np.mat([float(e) for e in row[10:19]]).reshape([3,3]),
            'C': np.mat([float(e) for e in row[19:22]]).reshape([3,1]),
            'image': row[22],
            # ignoring the flow fields
        })

for camera in cameras:
#     print(camera)
    camera['t'] = -camera['R'] * camera['C']

In [ ]:
pointcloud_csv = cache_path + r"\PointCloud.csv"
points = []

with open(pointcloud_csv, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        points.append({
            'index': int(row[0]),
            'X': np.mat([float(e) for e in row[1:4]]).reshape([3,1]),
            'colour': np.mat([float(e) for e in row[4:7]]).reshape([3,1]),
            'error': float(row[7])
        })

# for point in points:
#     print(point)

In [ ]:
if not os.path.exists("scratch"):
    os.makedirs("scratch")

for camera in cameras:
    print("Reading image " + camera['image'] + " ...")
    img = imageio.imread(camera['image'])

    for point in points[:2000]:
        point_cam = camera['R'] * point['X'] + camera['t']

        ## Convert from camera coords to pixel coords using OpenVSLAM code:
        ## https://github.com/xdspacelab/openvslam/blob/master/src/openvslam/camera/equirectangular.h
        bearing = point_cam / np.linalg.norm(point_cam)
#         print(f"bearing^T = {bearing.T}")

        ## convert to unit polar coordinates
        latitude = -np.arcsin(bearing[1])
        longitude = np.arctan2(bearing[0], bearing[2]);
#         print(f"latitude = {latitude}")
#         print(f"longitude = {longitude}")

        ## convert to pixel image coordinated
        x = img.shape[1] * (0.5 + longitude / (2.0 * np.pi))
        y = img.shape[0] * (0.5 - latitude / np.pi);
#         print(f"(x, y) = ({x}, {y})")

        ## round to nearest pixel
        x = int(x + 0.5)
        y = int(y + 0.5)

        ## mark the point
        img[y-7:y+8,x-7:x+8,:] = 255
        img[y-4:y+5,x-4:x+5,:] = 0

    ## save the image
    imageio.imwrite(f"scratch/{camera['frame']}.jpg", img)

In [ ]:
## Rename images sequentially to turn into video with ffmpeg
for i, f in enumerate(os.listdir('scratch'), start=1):
    os.rename(os.path.join('scratch', f), 'scratch/renamed-%04d.jpg' % i)

In [ ]:
## Run this manually to create a video:
#> ffmpeg -i scratch/renamed-%04d.jpg -c:v libx264 -crf 19 -level 3.1 -preset slow -tune film -movflags faststart -filter:v scale=-1:960 -sws_flags lanczos -threads 0 -nostdin output.mp4